## Action2：使用Google Colab编辑器，对MovieLens数据集进行评分预测，计算RMSE（使用funkSVD, BiasSVD，SVD++）

In [ ]:
from google.colab import drive
import os

In [ ]:
drive.mount('/content/drive', force_remount=True)
os.chdir('/content/drive/My Drive/Colab Notebooks/BaiduML/Computational_AD/MovieLens/')

In [ ]:
!ls

In [ ]:
!pip install surprise

In [ ]:
from surprise import SVD, SVDpp
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import KFold
import pandas as pd
import math
import time

In [ ]:
time1=time.time()

In [ ]:
# 数据读取
reader = Reader(line_format='user item rating timestamp', sep=',', skip_lines=1)
data = Dataset.load_from_file('ratings.csv', reader=reader)
train_set = data.build_full_trainset()

方法思想：分别在数据集上采用k折交叉验证的方式，训练和检验 FunkSVD, BiasSVD 和 SVD++ 三种SVD矩阵分解方法的预测准确率。然后选取其中平均RMSE结果最低的模型。最后将整个数据集当做训练集来重新训练表现最好的模型，并进行预测。

In [ ]:
# 使用funkSVD, BiasSVD 和 SVD++
# algo = SVD(biased=False)
algos = [SVD(biased=False), SVD(biased=True), SVDpp()]
algo_names = ['funkSVD', 'BiasSVD', 'SVD++']

In [ ]:
def search_model(algos, algo_names, k):
  assert(k>2)
  best_score = 1e10
  best_algo = None
  best_name = None
  for algo, algo_name in zip(algos, algo_names):
    # 定义K折交叉验证迭代器
    kf = KFold(n_splits=k)
    score = 0.0
    for trainset, testset in kf.split(data):
      # 训练并预测
      algo.fit(trainset)
      predictions = algo.test(testset)
      # 计算RMSE
      score += accuracy.rmse(predictions, verbose=True)
    if best_score > score:
      best_score = score
      best_algo = algo
      best_name = algo_name
  return best_algo, best_name, best_score
      

In [ ]:
best_algo, best_name, best_score = search_model(algos, algo_names, 3)
print('The best algorithm is {} with average k_fold rmse {}'.format(best_name, best_score))
# 在整个训练集上重新训练模型
best_algo.fit(train_set)

In [ ]:
uid = str(196)
iid = str(302)
# 输出uid对iid的预测结果
pred = best_algo.predict(uid, iid, r_ui=4, verbose=True)
time2=time.time()
print(time2-time1)